In [10]:
# System imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import panel as pn
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets
from string import digits
import csv
import json
import numpy as np
import dateparser
import panel as pn

pn.extension()

%matplotlib inline


In [11]:
# Local imports
import sys
sys.path.append("../lib2")

from Constants import Constants
from ProcessingTools import DateProcessingTools


In [12]:
# Construct the tools
debug_level = 0
constants = Constants()
tool_data_processing = DateProcessingTools(debug_level)


In [13]:
# Import all preprocessed data
atlas           = pd.read_pickle(constants.PREPROCESSED_ATLAS_FILE_PATH)
forecast        = pd.read_pickle(constants.PREPROCESSED_FORECAST_DATA_FILE_PATH)
revenue2020     = pd.read_pickle(constants.PREPROCESSED_REVENUE2020_FILE_PATH)
revenue2020A    = pd.read_pickle(constants.PREPROCESSED_REVENUE2020A_FILE_PATH)


In [14]:
atlas

,ID,Number of Users,Invoice Date,Invoice #,Year,Month,Invoice Amount,Subscription,2021 Prediction,Lost Customer,Customers Status,Account Code,Address,State,Lat,Long,Service Start,Service End,Subscription Duration
Customers,,,,,,,,,,,,,,,,,,,
Alabama A&M University-College of Education,1,246.0,2018-03-23 00:00:00-04:00,105,2018,March,5460.0,3 Years,"$1,820.00",NaN,New,4700-0-00-00000-00-0000,"4900 Meridian St N, Huntsville, AL 35811",AL,34.785061,-86.573280,2018-03-21 00:00:00-04:00,2021-03-31 00:00:00-04:00,1095 days
Albertus Magnus College,2,267.0,2018-05-16 00:00:00-04:00,127,2018,May,5670.0,3 Years,"$1,890.00",NaN,New,4700-0-00-00000-00-0000,"700 Prospect St, New Haven, CT 06511",CT,41.331299,-72.921440,2018-09-01 00:00:00-04:00,2021-08-31 00:00:00-04:00,1095 days
Albuquerque Public Schools,3,77.0,2018-11-20 00:00:00-05:00,161,2018,November,2270.0,1 Year,NaN,2019.0,New,4700-0-00-00000-00-0000,"Po Box 25704 Albuquerque, NM 87125",NM,35.092410,-106.642950,2018-11-19 00:00:00-05:00,2019-11-30 00:00:00-05:00,365 days
Alcorn State University,4,20.0,2017-10-31 00:00:00-04:00,NaN,2017,October,1700.0,1 Year,NaN,2018.0,New,4700-0-00-00000-00-0000,"1000 Asu Dr, Lorman, MS 39096",MS,31.875710,-91.141739,2017-08-31 00:00:00-04:00,2018-08-31 00:00:00-04:00,365 days
Alderson Broaddus University,5,50.0,2020-10-07 00:00:00-04:00,NB100820K,2020,October,500.0,3 Months,$500.00,NaN,New,4700-0-00-00000-00-0000,"101 College Hill Rd, Philippi, WV 26416",WV,39.160261,-80.049048,2020-10-02 00:00:00-04:00,2020-12-31 00:00:00-05:00,90 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Winthrop University,403,375.0,2020-09-08 00:00:00-04:00,NB090820S,2020,September,937.5,3 Months,NaN,NaN,Returning,4700-0-00-00000-00-0000,"701 Oakland Ave, Rock Hill, SC 29730",SC,34.938466,-81.029872,2020-09-01 00:00:00-04:00,2020-11-30 00:00:00-05:00,90 days
Winthrop University,403,100.0,2020-10-13 00:00:00-04:00,NB101320C,2020,October,1000.0,1 Year,"$1,000.00",NaN,Returning,4700-0-00-00000-00-0000,"701 Oakland Ave, Rock Hill, SC 29730",SC,34.938466,-81.029872,2020-09-30 00:00:00-04:00,2021-09-30 00:00:00-04:00,365 days
Yakima Valley College,404,56.0,2020-09-21 00:00:00-04:00,NB092120A,2020,September,1030.0,6 Months,"$1,030.00",NaN,New,4700-0-00-00000-00-0000,"PO Box 22520, Yakima, WA 99807-2520",WA,-35.001338,117.875053,2020-09-18 00:00:00-04:00,2021-03-18 00:00:00-04:00,180 days


In [15]:
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(mapbox_token)

In [16]:
total_data = atlas.loc[:, ["Number of Users", "Invoice Amount"]].groupby(["Customers"]).sum()
# total_data.dropna(inplace=True)
total_data

atlas_with_totals = pd.concat([atlas.drop(columns=["Number of Users", "Invoice Amount"]), total_data], axis="columns", join="inner")

In [17]:
px.set_mapbox_access_token(mapbox_token)
map = px.scatter_mapbox(
    atlas_with_totals.reset_index(),
    lat="Lat",
    lon="Long",
    # size="Invoice Amount",
    size="Invoice Amount",
    color="Customers",
    width=1800,
    height=900,
    zoom=4,
    mapbox_style="open-street-map"  # Options: [ "open-street-map, white-bg, carto-positron, carto-darkmatter, stamen-terrain, stamen-toner, stamen-watercolor" ]
)
map.show()


In [18]:
# Group by year and neighborhood and then create a new dataframe of the mean values
atlas_with_totals["ServiceStart_Year"] = atlas_with_totals["Service Start"].apply(lambda x: x.year)
atlas_with_totals_grouped = atlas_with_totals.groupby(["ServiceStart_Year", "State"]).sum()
atlas_with_totals_grouped.head(5)

def generate_map(state):
    data = atlas_with_totals.reset_index().set_index("State").loc[state]
    map = px.scatter_mapbox(
        data.reset_index(),
        lat="Lat",
        lon="Long",
        size="Invoice Amount",
        # size="Number of Users",
        color="Customers",
        width=1800,
        height=900,
        zoom=4,
        mapbox_style="open-street-map"  # Options: [ "open-street-map, white-bg, carto-positron, carto-darkmatter, stamen-terrain, stamen-toner, stamen-watercolor" ]
    )
    return map.show()

interact(generate_map, state=constants.DROP_DOWNS_STATE_ABBREVIATION_LIST)

Column
    [0] Column
        [0] Select(name='state', options=['AL', 'AK', 'AZ', ...], value='AL')
    [1] Row
        [0] Str(None, name='interactive01459')